### 第十九章 马尔可夫链蒙特卡罗方法

In [21]:
import numpy as np
import matplotlib.pyplot as plt


np.random.seed(42)  # 固定随机种子，结果可复现
theta0 = 0.5        # 初始值
sigma = 0.2         # 提议分布的标准差
n_burn = 100000    # 预热期次数
n_total = 300000  # 总采样次数

def target(theta):
    if theta < 0 or theta > 1:
        return 0 
    return theta**4 * (1-theta)**6

# Metropolis-Hastings采样
theta_chain = np.zeros(n_total)
theta_chain[0] = theta0

for t in range(1, n_total):
    theta_t_1 = theta_chain[t-1]
    # 从提议分布采样候选样本（截断正态，超出[0,1]则重采）
    while True:
        theta_star = np.random.normal(theta_t_1, sigma)
        if 0 <= theta_star <= 1:
            break
    # 计算接受概率
    alpha = min(1, target(theta_star) / target(theta_t_1))
    # 计算是否拒绝
    u = np.random.uniform(0, 1)
    if u <= alpha:
        theta_chain[t] = theta_star
    else:
        theta_chain[t] = theta_t_1


theta_eff = theta_chain[n_burn:]
mh_mean = np.mean(theta_eff)
mh_var = np.var(theta_eff, ddof=1) 

a, b = 5, 7
theo_mean = a / (a + b)
theo_var = (a * b) / ((a + b)**2 * (a + b + 1))

# ---------------------- 7. 结果输出 ----------------------
print(f"MH采样均值：{mh_mean:.6f}  MH采样方差：{mh_var:.6f}")
print(f"理论后验均值：{theo_mean:.6f} 理论后验方差：{theo_var:.6f}")


MH采样均值：0.421748  MH采样方差：0.017870
理论后验均值：0.416667 理论后验方差：0.018697


In [ ]:
def log_p_theta_given_eta(theta, eta):
    """p(theta|eta,y)的对数形式，theta∈(0,1-eta)"""
    if theta <= 0 or theta >= 1 - eta:
        return -np.inf
    p1 = theta/4 + 1/8
    p2 = theta/4
    p5 = (1 - theta - eta)/2
    return 14*np.log(p1) + np.log(p2) + 5*np.log(p5)

def log_p_eta_given_theta(eta, theta):
    """p(eta|theta,y)的对数形式，eta∈(0,1-theta)"""
    if eta <= 0 or eta >= 1 - theta:
        return -np.inf
    p3 = eta/4
    p4 = eta/4 + 3/8
    p5 = (1 - theta - eta)/2
    return np.log(p3) + np.log(p4) + 5*np.log(p5)

# Metropolis-Hastings抽样函数
def mh_sampler(log_p, x0, bounds, n_samples=1, step=0.01):
    x = x0
    samples = []
    for _ in range(n_samples):
        # 建议分布 均匀分步U(x-step, x+step)
        x_proposal = x + np.random.uniform(-step, step)
        # 计算接受概率
        log_p_curr = log_p(x)
        log_p_prop = log_p(x_proposal)
        accept_prob = np.min([1, np.exp(log_p_prop - log_p_curr)])
        # 拒绝采样
        if np.random.uniform(0,1) < accept_prob:
            x = x_proposal
        samples.append(x)
    return np.array(samples)

# 吉布斯抽样
def gibbs_sampler(n_total, n_burn, theta0, eta0, step_theta=0.005, step_eta=0.005):
    theta_samples = [theta0]
    eta_samples = [eta0]
    for t in range(n_total):
        # 固定eta，抽theta
        eta_curr = eta_samples[-1]
        log_p_theta = lambda x: log_p_theta_given_eta(x, eta_curr)
        theta_new = mh_sampler(log_p_theta, theta_samples[-1], (0, 1-eta_curr), n_samples=10, step=step_theta)[-1]
        # 固定theta，抽eta
        log_p_eta = lambda x: log_p_eta_given_theta(x, theta_new)
        eta_new = mh_sampler(log_p_eta, eta_samples[-1], (0, 1-theta_new), n_samples=10, step=step_eta)[-1]

        theta_samples.append(theta_new)
        eta_samples.append(eta_new)
    # 舍去燃烧期，返回有效样本
    theta_eff = np.array(theta_samples[n_burn:])
    eta_eff = np.array(eta_samples[n_burn:])
    return theta_eff, eta_eff


np.random.seed(42) 
n_total = 100000    
n_burn = 20000       
theta0, eta0 = 0.2, 0.2  # 初始值
theta_eff, eta_eff = gibbs_sampler(n_total, n_burn, theta0, eta0, step_theta=0.005, step_eta=0.005)


theta_mean = np.mean(theta_eff)
theta_var = np.var(theta_eff)
eta_mean = np.mean(eta_eff)
eta_var = np.var(eta_eff)

# 输出结果
print(f"theta的均值：{theta_mean:.4f}，方差：{theta_var:.6f}")
print(f"eta的均值：{eta_mean:.4f}，方差：{eta_var:.6f}")


theta的均值：0.5032，方差：0.017117
eta的均值：0.1268，方差：0.006818
